In [1]:
### This code is based on two asset model from https://github.com/shade-econ/sequence-jacobian
### If you have any questions, feel free to contact me by: ekaterina@shabalins.com

### loading directories
import numpy as np
import matplotlib.pyplot as plt

from sequence_jacobian import simple, solved, combine, create_model, estimation  # functions
from sequence_jacobian import grids, hetblocks   

In [2]:
### solved blocks
@solved(unknowns={'pi': (-0.1, 0.1)}, targets=['nkpc'], solver="brentq")
def pricing_solved(pi, mc, r, Y, kappap, mup):
    nkpc = kappap * (mc - 1/mup) + Y(+1) / Y * (1 + pi(+1)).apply(np.log) / \
           (1 + r(+1)) - (1 + pi).apply(np.log)
    return nkpc

@solved(unknowns={'p': (5, 15)}, targets=['equity'], solver="brentq")
def arbitrage_solved(div, p, r):
    equity = div(+1) + p(+1) - p * (1 + r(+1))
    return equity

In [3]:
### production blocks
@simple
def labor(Y, w, K, Z, alpha):
    N = (Y / Z / K(-1) ** alpha) ** (1 / (1 - alpha))
    mc = w * N / (1 - alpha) / Y
    return N, mc


@simple
def investment(Q, K, r, N, mc, Z, delta, epsI, alpha):
    inv = (K / K(-1) - 1) / (delta * epsI) + 1 - Q
    val = alpha * Z(+1) * (N(+1) / K) ** (1 - alpha) * mc(+1) -\
        (K(+1) / K - (1 - delta) + (K(+1) / K - 1) ** 2 / (2 * delta * epsI)) +\
        K(+1) / K * Q(+1) - (1 + r(+1)) * Q
    return inv, val


production = combine([labor, investment])                              # create combined block
production_solved = production.solved(unknowns={'Q': 1., 'K': 10.},    # turn it into solved block
                                      targets=['inv', 'val'],
                                      solver='broyden_custom')

In [4]:
### heterogeneous households

def make_grids(bmax, amax, kmax, nB, nA, nK, nZ, rho_z, sigma_z):
    b_grid = grids.agrid(amax=bmax, n=nB)
    a_grid = grids.agrid(amax=amax, n=nA)
    k_grid = grids.agrid(amax=kmax, n=nK)[::-1].copy()
    e_grid, _, Pi = grids.markov_rouwenhorst(rho=rho_z, sigma=sigma_z, N=nZ)
    return b_grid, a_grid, k_grid, e_grid, Pi


def income(e_grid, tax, w, N):
    z_grid = (1 - tax) * w * N * e_grid
    return z_grid

hh = hetblocks.hh_twoasset.hh
hh_ext = hh.add_hetinputs([income, make_grids])

In [5]:
### additional blocks and creation of the model

import sequence_jacobian.examples.two_asset as m

blocks = [hh_ext, production_solved, pricing_solved, arbitrage_solved,
          m.dividend, m.taylor, m.fiscal, m.share_value,
          m.finance, m.wage, m.union, m.mkt_clearing]

hank = create_model(blocks, name='Two-Asset HANK')

In [6]:
### steady state model
blocks_ss = [hh_ext, m.partial_ss, m.union_ss,
             m.dividend, m.taylor, m.fiscal, m.share_value, m.finance, m.mkt_clearing]

hank_ss = create_model(blocks_ss, name='Two-Asset HANK SS')

In [7]:
### calibration
#calibration = {'Y': 1., 'N': 1.0, 'K': 10., 'r': 0.0125, 'rstar': 0.0125, 'tot_wealth': 14,
#               'delta': 0.02, 'pi': 0., 'kappap': 0.1, 'muw': 1.1, 'Bh': 1.04, 'Bg': 2.8,
#               'G': 0.2, 'eis': 0.5, 'frisch': 1., 'chi0': 0.25, 'chi2': 2, 'epsI': 4,
#               'omega': 0.005, 'kappaw': 0.1, 'phi': 1.5, 'nZ': 3, 'nB': 50, 'nA': 70, 'nK': 50,
#               'bmax': 50, 'amax': 4000, 'kmax': 1, 'rho_z': 0.966, 'sigma_z': 0.92}

calibration = {'Y': 1., 'N': 1.0, 'K': 10., 'r': 0.0125, 'rstar': 0.0125, 'tot_wealth': 14,
               'delta': 0.02, 'pi': 0., 'kappap':  0.030, 'muw': 1.1, 'Bh': 1.04, 'Bg': 2.8,
               'G': 0.2, 'eis': 0.5, 'frisch': 1., 'chi0': 0.25, 'chi2': 2, 'epsI': 0.502,
               'omega': 0.005, 'kappaw': 0.011, 'phi': 1.297, 'nZ': 3, 'nB': 50, 'nA': 70, 'nK': 50,
               'bmax': 50, 'amax': 4000, 'kmax': 1, 'rho_z': 0.966, 'sigma_z': 0.92}

unknowns_ss = {'beta': 0.976, 'chi1': 6.5}
targets_ss = {'asset_mkt': 0., 'B': 'Bh'}

cali = hank_ss.solve_steady_state(calibration, unknowns_ss, targets_ss, solver='broyden_custom')

In [8]:
### steady state and dynamics
ss =  hank.steady_state(cali) # ss

exogenous = ['Z', 'G', 'rstar']
unknowns = ['r', 'w', 'Y']
targets = ['asset_mkt', 'fisher', 'wnkpc']
T = 300

J_ha = hh_ext.jacobian(ss, inputs=['N', 'r', 'ra', 'rb', 'tax', 'w'], T=T)
G = hank.solve_jacobian(ss, unknowns, targets, exogenous, T=T, Js={'hh': J_ha}) # dynamics

In [9]:
### analytical second moments
#sigma_persist = 0.4582
#rho = 0.9577
sigma_persist = 0.072 # from estimated two-asset
rho =  0.951 # from estimated two-asset
dZ = rho**(np.arange(T))
dY, dC, dI, dw = G['Y']['Z'] @ dZ, G['C']['Z'] @ dZ, G['I']['Z'] @ dZ, G['w']['Z'] @ dZ
dX1 = np.stack([dZ, dY, dC, dI, dw], axis=1)
Sigma = estimation.all_covariances(dX1[:, :, np.newaxis], sigma_persist)
sd = np.sqrt(np.diag(Sigma[0, ...]))
print('Z', 'Y', 'C', 'I', 'w')
print(sd / sd[1]) # relative to Y

Z Y C I w
[0.78128332 1.         0.75948078 0.22915784 0.76458717]


In [10]:
### function for numerical simulation
def simulation(endogenous, exogenous, T, se, rho, G, ss): # only for linear solutions!
    
    # input variables:
    # endogenous - variables which trajectories are simulated
    # exogenous - shocks
    # T - number of time periods for simulation (should be equal to row dimention of G)
    # se - vector of std of all shocks
    # rho - vector of persistences of the shocks
    # G - model jacobians
    # ss - model steady state
    
    # output variables:
    # end_variables - simulated trajectories
    
    i = 0 # iterator through shocks
    end_variables = np.zeros((len(endogenous), T)) # collects paths of endogenous variables
    for shock in exogenous: # iteration through shocks
        dshock = np.random.normal(loc=0, scale=se[i], size=(T,)) # generate shocks
        j = 0 # itereator through endogeneous variables
        for variable in endogenous: # iteration through endogeneous variables
            for t in np.arange(0, len(dshock), 1): # consider each shock at time = t as unexpected and iterate through all time periods/shocks
                if rho[i] == 0: # no persistence
                    coefs = np.zeros((len(dshock),))
                    coefs[t:] = G[variable][shock][:T-t, 0] # IRF coefficients from the on impact till T-t; from the first column to extract current and not news shocks
                    end_variables[j] += coefs * dshock[t] * ss[variable] # add responses to shock "shock" that occurs at time t
                else:
                    shock_pers = np.zeros((len(dshock),))
                    for jj in np.arange(t, T, 1):
                        shock_pers[jj] = dshock[jj] * (t == jj) + shock_pers[jj-1] * rho[i] # accumulation due to persistence
                    coefs = np.block([[np.zeros((t,T))], [np.zeros((T-t, t)), G[variable][shock][:T-t, :T-t]]]) # IRF coefficients from the on impact till T-t; from the first column to T-t column to extract current and news shocks (persistence is taken into account by the agents once the shock occured)
                    #end_variables[j] += coefs @ shock_pers * ss[variable]
                    end_variables[j] += coefs @ shock_pers# add responses to shock "shock" that occurs at time t
            j += 1
        i += 1
    return end_variables

In [11]:
### numerical moments
exogenous = ['Z']
endogenous = ['Y', 'C', 'I', 'w'] # variables, which trajectories are simulated
#se = np.array([0.4582]) # from replication of SW2007 in MMB
#rho = np.array([0.9577]) # from replication of SW2007 in MMB
se = np.array([0.072]) # from estimated two-asset
rho =  np.array([0.951]) # from estimated two-asset

std_all = np.zeros((150, 4))
mean_all = np.zeros((150, 4))
for i in np.arange(0, 150, 1):
    simulated_variables = simulation(endogenous, exogenous, T, se, rho, G, ss) # trajectories of all endogenous variables in deviations from steady states (exogenous shocks are treated as sequence of unexpected shocks)
    std_all[i] = np.std(simulated_variables[:, 50:250], axis=1) # std of all endogenous variables (in the same order as endogenous)
    mean_all[i] = np.mean(simulated_variables[:, 50:250], axis=1)
    #print(i) # takes 7 minutes
simulated_sd = np.mean(mean_all, axis=0)
print('Y', 'C', 'I', 'w')
print(simulated_sd / simulated_sd[0]) # relative to Y

Y C I w
[1.         0.74632924 0.24163601 0.82783314]


In [12]:
### numerical moments with three shocks (monetary policy, government spending and TFP)
#se = np.array([0.4582, 0.5291,  0.2449]) # from replication of SW2007 in MMB
#rho = np.array([0.9577, 0.9767, 0.15]) # from replication of SW2007 in MMB

se = np.array([0.072, 0.643,  0.663]) # from estimated two-asset
rho = np.array([0.951, 0.952, 0.737]) # from estimated two-asset

std_all = np.zeros((150, 4))
mean_all = np.zeros((150, 4))
for i in np.arange(0, 150, 1):
    simulated_variables = simulation(endogenous, exogenous, T, se, rho, G, ss) # trajectories of all endogenous variables in deviations from steady states (exogenous shocks are treated as sequence of unexpected shocks)
    std_all[i] = np.std(simulated_variables[:, 50:250], axis=1) # std of all endogenous variables (in the same order as endogenous)
    mean_all[i] = np.mean(simulated_variables[:, 50:250], axis=1)
    #print(i)
simulated_sd = np.mean(mean_all, axis=0)
print('Y', 'C', 'I', 'w')
print(simulated_sd / simulated_sd[0]) # relative to Y

Y C I w
[1.         0.69824481 0.29039874 0.89410847]


In [13]:
### numerical moments with seven shocks (as in SW 2007)

# adding discount factor shocks
@simple
def discount_factor(eps_beta): # add this block to shock beta and get risk-premium shock as in SW2007
    beta = 0.9762739008880043 + eps_beta
    return beta

# steady state with eps_beta = 0
ss['eps_beta'] =  0

# dynamics with discount factor block
T = 1000
blocks = [hh_ext, production_solved, pricing_solved, arbitrage_solved,
          m.dividend, m.taylor, m.fiscal, m.share_value,
          m.finance, m.wage, m.union, m.mkt_clearing, discount_factor]

hank = create_model(blocks, name='Two-Asset HANK')

exogenous = ['Z', 'eps_beta', 'G', 'epsI', 'rstar', 'mup', 'muw']

J_ha = hh_ext.jacobian(ss, inputs=['N', 'r', 'ra', 'rb', 'tax', 'w'], T=T)
G = hank.solve_jacobian(ss, unknowns, targets, exogenous, T=T, Js={'hh': J_ha}) # dynamics

# simulations
#se = np.array([0.4582, 0.2400 / (1 + ss['r']) ** 2, 0.5291, 0.4526, 0.2449, 0.1410 / ss['mup'] ** 2 * ss['kappap'], 0.2446 / ss['muw'] ** 2 * ss['kappaw']]) # from replication of SW2007 in MMB
#rho = np.array([0.9577, 0.2194, 0.9767, 0.7113, 0.15, 0.8895, 0.9688]) # from replication of SW2007 in MMB
se = np.array([0.072, 0.096, 0.643, 0.413, 0.663, 0.049, 0.155]) # from replication of SW2007 in MMB
rho = np.array([0.951, 0.943, 0.952, 0.656, 0.737, 0.891, 0.586]) # from replication of SW2007 in MMB


std_all = np.zeros((100, 4))
mean_all = np.zeros((100, 4))
for i in np.arange(0, 100, 1):
    simulated_variables = simulation(endogenous, exogenous, T, se, rho, G, ss) # trajectories of all endogenous variables in deviations from steady states (exogenous shocks are treated as sequence of unexpected shocks)
    std_all[i] = np.std(simulated_variables[:, 50:250], axis=1) # std of all endogenous variables (in the same order as endogenous)
    mean_all[i] = np.mean(simulated_variables[:, 50:250], axis=1)
    if (i% 10) == 0:
        print(i) # takes 6 hours
simulated_sd = np.mean(mean_all, axis=0)
print('Y', 'C', 'I', 'w')
print(simulated_sd / simulated_sd[0]) # relative to Y

0
10
20
30
40
50
60
70
80
90
Y C I w
[1.         0.54645322 0.14204396 0.34444066]
